In [5]:
import pandas as pd
import numpy as np

In [6]:
# --- Configurações ---
arquivo_topics = 'topics.xlsx'
arquivo_systems = 'systems_em_mercado.xlsx'
max_score_individual = 3 # A pontuação máxima que um item pode ter em uma categoria
max_score_combinado = max_score_individual * 2 # A pontuação máxima da combinação (3 + 3 = 6)

In [7]:
# --- Listas de Atributos (Ajuste se os nomes das suas colunas forem diferentes) ---
# É importante separar para podermos combinar um de cada tipo
generos = ['Action', 'Adventure', 'RPG', 'Simulation', 'Strategy', 'Casual']
faixas_etarias = ['Y', 'E', 'M'] # Y: Young, E: Everyone (Livre), M: Mature

In [8]:
# --- Carregamento dos Dados ---
try:
    df_topics = pd.read_excel(arquivo_topics, index_col=0)
    df_systems = pd.read_excel(arquivo_systems, index_col=0)
except FileNotFoundError:
    print(f"AVISO: Um ou ambos os arquivos Excel ('{arquivo_topics}', '{arquivo_systems}') não foram encontrados.")
    print("Criando arquivos de exemplo para demonstração...")

    # Dados de exemplo para topics.xlsx
    data_topics = {
        'Topic': ['Abstract', 'Airplane', 'Aliens', 'Alternate History', 'Assassin', 'Fantasy'],
        'Action': [2, 3, 3, 3, 3, -1],
        'Adventure': [3, -3, 1, 1, -2, 3],
        'RPG': [-3, 1, 3, 3, 3, 3],
        'Simulation': [-3, 3, -3, 1, 1, 0],
        'Strategy': [1, 3, 2, 2, -3, 2],
        'Casual': [-3, 3, -2, -3, -3, 1],
        'Y': [1, 3, 2, -3, -3, 2],
        'E': [2, 3, 3, 3, 1, 3],
        'M': [3, 2, 3, 3, 3, 3]
    }
    df_topics = pd.DataFrame(data_topics).set_index('Topic')
    df_topics.to_excel(arquivo_topics)
    print(f"Arquivo '{arquivo_topics}' de exemplo criado.")

    # Dados de exemplo para systems.xlsx (com 'System' como nome do índice)
    data_systems = {
        'System': ['PC', 'G64', 'TES', 'Master V', 'Gameling', 'Vena Gear', 'CloudPlay'],
        'Action': [2, 2, 1, 2, 1, 2, 3],
        'Adventure': [3, 3, -2, -2, -2, 1, 3],
        'RPG': [2, 2, 1, 1, 2, 1, 3],
        'Simulation': [3, 2, 1, 1, 2, 2, 1],
        'Strategy': [3, 3, -2, -2, -3, -3, 1],
        'Casual': [-3, -2, 3, 3, 3, 3, 2],
        'Y': [1, 1, 3, 2, 3, 2, 3],
        'E': [2, 2, -2, 3, 2, 3, 3],
        'M': [3, 3, 1, -2, -3, 1, 3]
    }
    df_systems = pd.DataFrame(data_systems).set_index('System')
    df_systems.to_excel(arquivo_systems)
    print(f"Arquivo '{arquivo_systems}' de exemplo criado.")
    print("\n--- Rodando com dados de exemplo ---")

# Renomear o índice do df_systems para 'System' para clareza consistente
df_systems = df_systems.rename_axis("System")

# --- Opcional: Exibir Tabelas Originais com Gradiente (para referência) ---
def aplicar_estilo_original(df, caption_text="Tabela Original"):
    return df.style.background_gradient(cmap='RdYlGn', vmin=-3, vmax=3)\
                   .set_caption(caption_text)\
                   .format("{:.0f}")

print("\n--- Tabela de Topics (Referência) ---")
display(aplicar_estilo_original(df_topics, "Topics - Pontuações Individuais"))

print("\n--- Tabela de Systems (Referência) ---")
display(aplicar_estilo_original(df_systems, "Systems - Pontuações Individuais"))





--- Tabela de Topics (Referência) ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
Topic,,,,,,,,,
Abstract,2,3,-3,-3,1,-3,1,2,3
Airplane,3,-3,1,3,3,3,3,3,2
Aliens,3,1,3,-3,2,-2,2,3,3
Alternate History,3,1,3,1,2,-3,-3,3,3
Assassin,3,-2,3,1,-3,-3,-3,1,3
Business,-3,1,1,3,3,-3,2,3,-2
City,-2,-3,-2,3,3,-2,2,3,1
Colonization,-2,-3,-3,3,3,-2,-2,3,1
Comedy,-3,3,1,-3,-3,3,1,2,3



--- Tabela de Systems (Referência) ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
System,,,,,,,,,
PC,2,3,2,3,3,-3,1,2,3
GS,2,2,3,2,2,3,3,2,1
PPS,3,-2,3,1,1,1,1,2,3
gPhone,1,1,-2,2,-2,3,2,3,-3
gPad,1,2,-2,2,2,3,-2,3,-3
mPad,-2,2,1,2,-2,2,-2,2,1
OYA,2,-2,1,2,1,3,1,3,2
Swap,0,0,0,0,-2,0,0,0,1
mBox Next,2,2,2,1,-2,3,2,3,1


In [9]:
# --- Filtrar colunas para garantir que só usamos as existentes em ambos ---
# E que pertencem às nossas listas de generos e faixas_etarias
colunas_comuns = df_topics.columns.intersection(df_systems.columns)
generos_comuns = [g for g in generos if g in colunas_comuns]
faixas_etarias_comuns = [f for f in faixas_etarias if f in colunas_comuns]

if not generos_comuns:
    print("\nERRO CRÍTICO: Nenhuma coluna de 'Gênero' definida em 'generos' foi encontrada em ambos os arquivos. Verifique os nomes das colunas.")
    # exit() # Descomente para parar a execução se for crítico
if not faixas_etarias_comuns:
    print("\nERRO CRÍTICO: Nenhuma coluna de 'Faixa Etária' definida em 'faixas_etarias' foi encontrada em ambos os arquivos. Verifique os nomes das colunas.")
    # exit() # Descomente para parar a execução se for crítico

# Criar cópias filtradas para não modificar os DataFrames originais
df_topics_filtered = df_topics[list(set(generos_comuns + faixas_etarias_comuns))].copy() # Usar set para evitar duplicatas se um atributo estiver em ambas as listas
df_systems_filtered = df_systems[list(set(generos_comuns + faixas_etarias_comuns))].copy()

# --- Encontrar as Combinações Perfeitas ---
combinacoes_perfeitas = []

if generos_comuns and faixas_etarias_comuns: # Prosseguir apenas se tivermos categorias para combinar
    # Iterar por cada Topic
    for topic_name, topic_scores in df_topics_filtered.iterrows():
        # Iterar por cada System
        for system_name, system_scores in df_systems_filtered.iterrows():
            # Iterar por cada Gênero comum
            for genero in generos_comuns:
                # Iterar por cada Faixa Etária comum
                for faixa_etaria in faixas_etarias_comuns:

                    # Calcular o score combinado para o Gênero
                    score_combinado_genero = topic_scores[genero] + system_scores[genero]

                    # Calcular o score combinado para a Faixa Etária
                    score_combinado_faixa = topic_scores[faixa_etaria] + system_scores[faixa_etaria]

                    # Verificar se AMBOS os scores combinados atingem o máximo
                    if score_combinado_genero == max_score_combinado and score_combinado_faixa == max_score_combinado:
                        combinacoes_perfeitas.append({
                            'Tema (Topic)': topic_name,
                            'Gênero': genero,
                            'Plataforma (System)': system_name,
                            'Faixa Etária': faixa_etaria
                            # 'Score Gênero': score_combinado_genero, # Opcional para debug
                            # 'Score Faixa': score_combinado_faixa   # Opcional para debug
                        })
else:
    print("\nAVISO: Não foi possível prosseguir com a busca por combinações perfeitas devido à falta de colunas de gênero ou faixa etária comuns.")


# --- Exibir os Resultados Estilizados ---
print(f"\n--- Lista de Combinações com Pontuação Máxima ({max_score_combinado}) SIMULTÂNEA para Gênero E Faixa Etária ---")

if combinacoes_perfeitas:
    df_resultados = pd.DataFrame(combinacoes_perfeitas)
    # Reordenar colunas para o formato desejado
    df_resultados = df_resultados[['Tema (Topic)', 'Gênero', 'Plataforma (System)', 'Faixa Etária']]

    # Aplicar Estilos usando o .style do Pandas
    styled_df = df_resultados.style \
        .set_caption("🏆 Combinações Perfeitas Encontradas 🏆") \
        .hide(axis="index") \
        .set_table_styles([
            # Estilo para a tabela inteira
            {'selector': 'table',
             'props': [('border-collapse', 'collapse'),
                       ('border', '2px solid #4CAF50'),        # Borda verde escura
                       ('font-family', '"Segoe UI", Tahoma, Geneva, Verdana, sans-serif'), # Fonte moderna
                       ('width', '90%'),                     # Largura
                       ('margin-left', 'auto'),              # Centralizar
                       ('margin-right', 'auto'),
                       ('box-shadow', '0 4px 8px 0 rgba(0,0,0,0.1)') # Sombra sutil
                      ]},
            # Estilo para os cabeçalhos (th)
            {'selector': 'th',
             'props': [('background-color', '#4CAF50'),     # Fundo verde escuro
                       ('color', 'white'),                  # Texto branco
                       ('border', '1px solid #388E3C'),     # Borda interna um pouco mais escura que o fundo
                       ('padding', '12px 15px'),            # Espaçamento
                       ('text-align', 'center'),
                       ('font-weight', 'bold'),             # Negrito
                       ('font-size', '1.1em')
                      ]},
            # Estilo para as células de dados (td)
            {'selector': 'td',
             'props': [('border', '1px solid #ddd'),         # Borda cinza clara
                       ('padding', '10px 12px'),
                       ('text-align', 'left'),              # Alinhar texto à esquerda para melhor leitura dos nomes
                       ('color', '#333')                    # Cor do texto (cinza escuro, bom contraste com branco/cinza claro)
                      ]},
            # Estilo para linhas pares (efeito Zebra)
            {'selector': 'tbody tr:nth-child(even)',
             'props': [('background-color', '#f9f9f9')      # Cinza muito claro
                      ]},
            # Estilo para linhas ímpares (fundo branco padrão, mas explicitado aqui)
            {'selector': 'tbody tr:nth-child(odd)',
             'props': [('background-color', '#ffffff')      # Branco
                      ]},
            # Estilo para quando o mouse passa sobre uma linha (hover)
            {'selector': 'tbody tr:hover',
             'props': [('background-color', '#e8f5e9')      # Verde muito claro no hover
                      ]},
             # Estilo da Legenda (Caption)
             {'selector': 'caption',
              'props': [('caption-side', 'top'),
                        ('font-size', '1.4em'),
                        ('font-weight', 'bold'),
                        ('color', '#4CAF50'),                # Cor da legenda (verde)
                        ('margin-bottom', '20px'),
                        ('text-align', 'center')
                       ]}
        ])

    display(styled_df)
else:
    print("Nenhuma combinação atingiu a pontuação máxima simultaneamente para um Gênero e uma Faixa Etária com os dados atuais.")

print(f"\nLógica: Buscamos Topic+System onde (Score Topic Genero + Score System Genero = {max_score_combinado}) E (Score Topic Faixa + Score System Faixa = {max_score_combinado}).")


--- Lista de Combinações com Pontuação Máxima (6) SIMULTÂNEA para Gênero E Faixa Etária ---


Tema (Topic),Gênero,Plataforma (System),Faixa Etária
Abstract,Adventure,PC,M
Airplane,Casual,GS,Y
Airplane,Casual,gPhone,E
Airplane,Casual,gPad,E
Airplane,Casual,OYA,E
Airplane,Casual,mBox Next,E
Airplane,Action,Playsystem 5,E
Airplane,Simulation,Playsystem 5,E
Aliens,Action,PPS,M
Aliens,RPG,PPS,M



Lógica: Buscamos Topic+System onde (Score Topic Genero + Score System Genero = 6) E (Score Topic Faixa + Score System Faixa = 6).


In [11]:
# --- CÉLULA ANTERIOR (já executada) ---
# ... (todo o código para calcular combinacoes_perfeitas e criar df_resultados) ...
# df_resultados = pd.DataFrame(combinacoes_perfeitas)
# df_resultados = df_resultados[['Tema (Topic)', 'Gênero', 'Plataforma (System)', 'Faixa Etária']]
# ... (e as exibições anteriores)


# --- NOVA CÉLULA DO JUPYTER NOTEBOOK ---

# Verificar se df_resultados existe e não está vazio
if 'df_resultados' in globals() and not df_resultados.empty:
    print("\n--- Tópicos com Múltiplas Plataformas Compatíveis para o Mesmo Gênero e Faixa Etária ---")

    # 1. Agrupar e Contar
    # Agrupamos pelas três colunas chave e contamos quantas plataformas únicas existem para cada grupo.
    # Usamos .transform('size') para que o resultado da contagem seja transmitido de volta
    # para cada linha original do grupo.
    df_resultados['Contagem_Plataformas_Por_Grupo'] = df_resultados.groupby(
        ['Tema (Topic)', 'Gênero', 'Faixa Etária']
    )['Plataforma (System)'].transform('size') # 'size' conta o número de linhas em cada grupo

    # 2. Filtrar os grupos que têm mais de uma plataforma
    df_multiplas_plataformas = df_resultados[df_resultados['Contagem_Plataformas_Por_Grupo'] > 1].copy()

    # 3. Remover a coluna de contagem auxiliar (opcional, para limpeza)
    # Se quiser manter para verificar, comente a linha abaixo
    df_multiplas_plataformas.drop('Contagem_Plataformas_Por_Grupo', axis=1, inplace=True)

    # 4. Ordenar para melhor visualização (opcional, mas recomendado)
    if not df_multiplas_plataformas.empty:
        df_multiplas_plataformas_ordenado = df_multiplas_plataformas.sort_values(
            by=['Tema (Topic)', 'Gênero', 'Faixa Etária', 'Plataforma (System)'],
            ascending=[True, True, True, True]
        )
    else:
        df_multiplas_plataformas_ordenado = pd.DataFrame() # DataFrame vazio se nada for encontrado

    # 5. Aplicar Estilização
    if not df_multiplas_plataformas_ordenado.empty:
        # Usando uma nova cor para distinguir esta visualização (ex: roxo)
        styled_multiplas_plataformas = df_multiplas_plataformas_ordenado.style \
            .set_caption("🌟 Tópicos com Múltiplas Plataformas Perfeitas (Mesmo Gênero/Faixa) 🌟") \
            .hide(axis="index") \
            .set_table_styles([
                # Estilo para a tabela inteira
                {'selector': 'table',
                 'props': [('border-collapse', 'collapse'),
                           ('border', '2px solid #6f42c1'),        # Borda roxa
                           ('font-family', '"Segoe UI", Tahoma, Geneva, Verdana, sans-serif'),
                           ('width', '90%'),
                           ('margin-left', 'auto'),
                           ('margin-right', 'auto'),
                           ('box-shadow', '0 4px 8px 0 rgba(0,0,0,0.1)')
                          ]},
                # Estilo para os cabeçalhos (th)
                {'selector': 'th',
                 'props': [('background-color', '#6f42c1'),     # Fundo roxo
                           ('color', 'white'),
                           ('border', '1px solid #5a32a3'),     # Borda interna um pouco mais escura
                           ('padding', '12px 15px'),
                           ('text-align', 'center'),
                           ('font-weight', 'bold'),
                           ('font-size', '1.1em')
                          ]},
                # Estilo para as células de dados (td)
                {'selector': 'td',
                 'props': [('border', '1px solid #ddd'),
                           ('padding', '10px 12px'),
                           ('text-align', 'left'),
                           ('color', '#333')
                          ]},
                # Estilo para linhas pares (efeito Zebra)
                {'selector': 'tbody tr:nth-child(even)',
                 'props': [('background-color', '#f4f0f8')      # Roxo bem claro
                          ]},
                # Estilo para linhas ímpares (fundo branco padrão)
                {'selector': 'tbody tr:nth-child(odd)',
                 'props': [('background-color', '#ffffff')
                          ]},
                # Estilo para quando o mouse passa sobre uma linha (hover)
                {'selector': 'tbody tr:hover',
                 'props': [('background-color', '#e0d6ef')      # Roxo muito claro no hover
                          ]},
                 # Estilo da Legenda (Caption)
                 {'selector': 'caption',
                  'props': [('caption-side', 'top'),
                            ('font-size', '1.4em'),
                            ('font-weight', 'bold'),
                            ('color', '#6f42c1'),                # Cor da legenda (roxo)
                            ('margin-bottom', '20px'),
                            ('text-align', 'center')
                           ]}
            ])

        display(styled_multiplas_plataformas)
    else:
        print("Nenhum Tópico encontrado com múltiplas plataformas para a mesma combinação de Gênero e Faixa Etária.")

elif 'df_resultados' in globals() and df_resultados.empty:
    print("\nO DataFrame de resultados original está vazio. Nenhuma combinação perfeita foi encontrada na primeira execução.")
else:
    print("\nO DataFrame 'df_resultados' não foi encontrado. Execute as células anteriores primeiro.")


--- Tópicos com Múltiplas Plataformas Compatíveis para o Mesmo Gênero e Faixa Etária ---


Tema (Topic),Gênero,Plataforma (System),Faixa Etária
Airplane,Casual,OYA,E
Airplane,Casual,gPad,E
Airplane,Casual,gPhone,E
Airplane,Casual,mBox Next,E
Cooking,Casual,OYA,E
Cooking,Casual,gPad,E
Cooking,Casual,gPhone,E
Cooking,Casual,mBox Next,E
Farming,Casual,OYA,E
Farming,Casual,gPad,E
